<a href="https://colab.research.google.com/github/francesco-mollica/word2vec-entity_embeddings/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
base_dir_models = root_dir + "tesi_magistrale/datasets/models_w2v/"
names = {1:"balanced_ontonotes", 2:"ontonotes", 3:"bbn", 4:"figer", 5:"choi"}

Mounted at /content/drive/


In [2]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
            for line in open(base_dir + self.filename):
                yield line.split()

In [3]:
import numpy as np
from gensim.models import Word2Vec
def restrict_w2v(w2v,restricted_word_set):
    w2v_restricted = Word2Vec()
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.wv.vocab)):
        word = w2v.wv.index2entity[i]
        vec = w2v.wv.vectors[i]
        vocab = w2v.wv.vocab[word]
        vec_norm = w2v.wv.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            new_vectors_norm.append(vec_norm)

    
    w2v_restricted.wv.vocab = new_vocab
    w2v_restricted.wv.vectors = np.array(new_vectors)
    w2v_restricted.wv.index2entity = np.array(new_index2entity)
    w2v_restricted.wv.index2word = np.array(new_index2entity)
    w2v_restricted.wv.vectors_norm = np.array(new_vectors_norm)
    return w2v_restricted

In [4]:
for value in names.items():
  # extract restricted_word_list from types_counter dictionary
  import json
  types={}
  restricted_word_list=[]
  print(base_dir + "corpus_data_entire_"+value[1]+".txt")
  with open(base_dir + "types_counters_"+value[1]+".json") as j:
    types = json.load(j)
  for k, v in types.items():
    restricted_word_list.append(k)

  print("corpus_data_entire_" + value[1] + ".txt")
  sentences = MySentences("corpus_data_entire_" + value[1] + ".txt")
  
  w2v_model = Word2Vec(iter=1, min_count=5, seed=1234)
  w2v_model.build_vocab(sentences)
  w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
  # execute one time a most_similar otherwise vectors_norm in restrict_w2v should be equal to None
  w2v_model.wv.most_similar(positive=restricted_word_list[0])
  # restrict model
  
  w2v_restricted= restrict_w2v(w2v_model, restricted_word_list)
  # save
  w2v_model.save(base_dir_models + "word2vec_" + value[1] + ".model")
  w2v_model.save(base_dir_models + "word2vec_restricted_" + value[1] + ".model")

  del restricted_word_list
  del sentences
  del w2v_model

/content/drive/MyDrive/tesi_magistrale/datasets/corpus_data_folder/corpus_data_entire_bbn.txt
corpus_data_entire_bbn.txt
/content/drive/MyDrive/tesi_magistrale/datasets/corpus_data_folder/corpus_data_entire_figer.txt
corpus_data_entire_figer.txt


KeyboardInterrupt: ignored